In [23]:
data = Import["tables/salaries_2020-2025(clean).csv", "CSV"];
groupedData = Import["tables/salaries_2020-2025(grouped_log).csv", "CSV"];

In [25]:
data = Rest[data];
groupedData = Rest[groupedData];

In [27]:
graph1 = Graph[UndirectedEdge @@@ data];
graph2 = Graph[UndirectedEdge @@@ groupedData];

In [16]:
partition[g_] := Module[{vertex, u, v, c, list},
  vertex = VertexList[g]; 
  u = {};
  v = {};
  c = Complement[vertex, Join[u, v]]; (* Vértices que no están en u ni en v *)
  
  While[c != {}, (* Mientras haya vértices no asignados *)
    list = GraphDistance[g, c[[1]]]; (* Distancias de c[[1]] a todos los vértices *)
    
    (* Iteramos sobre la lista de distancias *)
    For[j = 1, j <= Length[list], j++,
      If[NumericQ[list[[j]]], (* Verificamos si la distancia es numérica *)
        If[EvenQ[list[[j]]], 
          AppendTo[u, vertex[[j]]], (* Si la distancia es par, lo asignamos a u *)
          AppendTo[v, vertex[[j]]]  (* Si la distancia es impar, lo asignamos a v *)
        ]
      ]
    ];
    
    c = Complement[vertex, Join[u, v]] (* Actualizamos c con los vértices no asignados *)
  ];
  
  {u, v} (* Devolvemos los dos conjuntos de vértices *)
]

In [35]:
projection[graph_, setUorV_] :=
  Module[{set, l,projectedGraph, list, list1, added, list2},
  
    set = Switch[setUorV,
      U, partition[graph][[1]],
      V, partition[graph][[2]]
    ];
    l = Length[set];
    projectedGraph = Graph[{}];
    (* Lista de vecinos de cada vértice del conjunto seleccionado *)
    list = AdjacencyList[graph, #] & /@ set;
    (* Itera sobre cada vértice en el conjunto seleccionado (set). *)
    For[i = 1, i <= l, i++,
      
      list1 = list[[i]];
      added = False;
      
      (* Verifica si hay intersección entre vecinos de i y j *)
      For[j = i + 1, j <= l, j++,
        list2 = list[[j]];
        
          If[Intersection[list1, list2] != {},
            projectedGraph = EdgeAdd[projectedGraph, set[[i]] <-> set[[j]]];
            added = True
          ]
        
      ];
      
      (* Si el vértice no fue conectado a nadie, se agrega como nodo aislado *)
      If[!added,
        projectedGraph = VertexAdd[projectedGraph, set[[i]]]
      ]
    ];
    
    projectedGraph
  ]

In [30]:
partitionedSets = partition[graph2];
RandomSample[partitionedSets[[1]], 5]
RandomSample[partitionedSets[[2]], 5]
Length[partitionedSets[[1]]]
Length[partitionedSets[[2]]]

{21776.97 to 22464.13, 44495.64 to 45899.68, 269689.33 to 278199.25, 
 
>   31615.75 to 32613.37, 124039.31 to 127953.31}
{Postdoctoral Fellow, Backend Engineer, Data Integrity Analyst, 
 
>   Business Intelligence Manager, Data Visualization Specialist}
128
317

In [37]:
graphJob = projection[graph2, V]; (*GroupedJob*)